In [1]:
# Import system functions
import sys

In [2]:
# Import web/browser navigators that enable remote automated control of web browser
import time
import urllib
import requests
from splinter import Browser
from selenium import webdriver

In [3]:
# Import authentication information
from TDLogin import TDusername, TDpassword, TDconsumerkey

In [4]:
# Import Token Information
from Token import AccessToken, RefreshToken

In [5]:
# Assign Working Directory
sys.path.append(r'C:\Users\cinji\Documents\PythonAutoTrade')

In [6]:
# Open Chrome Browser

# Define path to Chrome Driver
executable_path = {'executable_path':r'C:\Users\cinji\Desktop\chromedriver_win32\chromedriver'}

# Set some default browser behaviors
options = webdriver.ChromeOptions()

# Make sure the window is maximized
options.add_argument("--start-maximized")

# Create a new browser object
browser = Browser('chrome', **executable_path, headless = False, options = options)

In [7]:
# Open TD Ameritrade App Developer Page

# Define the URL components
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_id = TDconsumerkey + '@AMER.OAUTHAP'
payload = {'response_type':'code', 'redirect_uri':'http://localhost', 'client_id':client_id}

# Build URL
completeURL = requests.Request(method, url, params = payload).prepare()
completeURL = completeURL.url

# Open Log-in page
browser.visit(completeURL)

# Defin the elements to pass through log-in form
payload = {'username':TDusername, 'password':TDpassword}

# Fill out each element in the log-in form
browser.find_by_id("username").first.fill(payload['username'])
browser.find_by_id("password").first.fill(payload['password'])
browser.find_by_id("accept").first.click()

# Click to accept terms and conditions on Authorization page
browser.find_by_id("accept").first.click()

# Provide page loading time
time.sleep(1)
newURL = browser.url

In [8]:
# Get browser URL and parse it to obtain access token
parseURL = urllib.parse.unquote(newURL.split('code=')[1])

# Close browser
browser.quit()

In [9]:
# Authentication Endpoint
# NOTE: This cell can ONLY BE DONE ONCE.  Otherwise, parseURL becomes invalid.

# Define access token endpoint
url = r'https://api.tdameritrade.com/v1/oauth2/token'

# Define access token headers
headers = {'Content-Type':"application/x-www-form-urlencoded"}

# Define access token payload
payload = {'grant_type':'authorization_code',
           'access_type':'offline',
           'code':parseURL,
           'client_id':client_id,
           'redirect_uri':'http://localhost'}

# Post data to get access token
authReply = requests.post(url, headers = headers, data = payload)

# Convert JSON string to dictionary
decodedURL = authReply.json()

In [10]:
# Parse retrieved access token
AccessToken = decodedURL['access_token']
RefreshToken = decodedURL['refresh_token']
headers = {'Authorization': "Bearer {}".format(AccessToken)}

In [11]:
# Accounts Endpoint
# Queries ALL Accounts

# Define accounts endpoint
endpoint = r"https://api.tdameritrade.com/v1/accounts"

# Make request
content = requests.get(url = endpoint, headers = headers)

# Convert to dictionary object
data = content.json()
TDAccountID = data[0]['securitiesAccount']['accountId']

In [12]:
# Single Account Endpoint
# Queries ONE Account

# Define accounts endpoint
endpoint = r"https://api.tdameritrade.com/v1/accounts/{}".format(TDAccountID)

# Make request
content = requests.get(url = endpoint, headers = headers)

# Convert to dictionary object
data = content.json()

In [13]:
TDAccountID

'861370305'

In [14]:
# Orders for Trading

# Create Saved Orders

# Define headers
header = {'Authorization':"Bearer {}".format(AccessToken),
          'Content-Type':"application/json"}

# Define the endpoint for saved orders, including AccountID
endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(TDAccountID)

# Define payload, in JSON format
payload = {
  "orderType": "MARKET",
  "session": "NORMAL",
  "duration": "DAY",
  "orderStrategyType": "SINGLE",
  "orderLegCollection": [
    {
      "instruction": "Buy",
      "quantity": 1,
      "instrument": {
        "symbol": "PINS",
        "assetType": "EQUITY"
      }
    }
  ]
}

# Make post
content = requests.post(url = endpoint, json = payload, headers = header)

# Show status code, want 200
content.status_code

400

In [15]:
# SAVED ORDERS ENDPOINT - POST

# define our headers
header = {'Authorization':"Bearer {}".format(AccessToken),
          "Content-Type":"application/json"}

# define the endpoint for Saved orders, including your account ID
endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(TDAccountID)

# define the payload, in JSON format
payload = {'orderType':'MARKET',
           'session':'NORMAL',
           'duration':'DAY',
           'orderStrategyType':'SINGLE',
           'orderLegCollection':[{'instruction':'Buy','quantity':1,'instrument':{'symbol':'PINS','assetType':'EQUITY'}}]}


# make a post, NOTE WE'VE CHANGED DATA TO JSON AND ARE USING POST
content = requests.post(url = endpoint, json = payload, headers = header)

# show the status code, we want 200
content.status_code

400

In [16]:
# Refresh Access Token

# Define access token endpoint
url = r'https://api.tdameritrade.com/v1/oauth2/token'

# Define access token headers
headers = {'Content-Type':"application/x-www-form-urlencoded"}

# Define access token payload
payload = {'grant_type':'refresh_token',
           'refresh_token':RefreshToken,
           'access_type':'offline',
           'code':AccessToken,
           'client_id':client_id,
           'redirect_uri':'http://localhost'}

# Post data to get access token
authReply = requests.post(url, headers = headers, data = payload)

# Convert JSON string to dictionary
decodedURL = authReply.json()
decodedURL

{'access_token': 'mS6E2VdPfIF3HaRe4auisdZhV8NAzwAXt6M9/ruea8B4W5lr9pH+6IGNeSthThEHy3qe67wgNOMZRwf108+tT2BDXUx0LPcPr77JEjosd0OWf2YhpO2q9dMAayVD2xX+6UZXgm2RCxGFGnEdSXK/xXIYeBE+NNJIXfHwRBNjq2K6BiTCSUxUX69z/27Y6LWpNlz0iauJE8NCxptzg2lMGhxYr/OFZb18nzUg1y4EpMQrG7ifaFd4MPPFpjN6XEN1jMqARPN/LczgWlycs45gXxMQP4WaIBqhy07Zg2y7ad+qQnUaOd6ps5UPOxKB1VW+imNbtGRB7lUy3JtorFErE2WzJcBJL2NgFdkTrvt3yMhfTy08eHW2GXLU961FpU1ap5ak3CZfa7n/aRF9Op+zLmCEdvFULcVABiAcntD+IkoPbHfu5LLzlWeZTskym8z+VWvX/awUhfnWdejjAYZ7rt4lWDQLejioSoMKIJ98vCB8EhVKqnHkVvky9JlJy4RfK5CL8AuHQlfBuzZb46gpBVA8nxG100MQuG4LYrgoVi/JHHvlVUrtekEf4rjCLf7Vcz+1vx9Ov4/rSIsx1h5G6H/uAwAWZ6Bd5KAU3GMlTglj65j0W2BkvJMeC3wBMMauuR2Ghis8w2F2n5zuPhG0ivnoVBJBAO4GFpyQX3yCBYbkor5NU1VAFonbfMsKf3jUf6Aaoyp3zW/e7bbL81VsF3GzDdfoHc0nBJxytFz98Z27Ek7nnihGWomy/pXXbRqrPPeWESKfuFfP7g/QIt9UzcW9sdRL3qMsHOdSeXwvSF3eoAP9vRvJvSYC7uVD+fqUX3jFLEwzRAZmm+QZpCy8wUj1oQ8q2rCGXBTa+FgIkS2bY0EgaqQSqDQBEC5bgJyb9zAw5zOSkVY6DVesSRJIV8ZA0FzivYeMYKNKKzkxzAN06wXsdoz5p0HSOB0KUMJsoNfj0MqnFP0rnwkNhuDbBY